In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import os
from glob import glob
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from google.colab import files  # Added for Colab file upload
import zipfile  # Added for ZIP extraction

# Create directories for storage
os.makedirs('checkpoints', exist_ok=True)
os.makedirs('tfrecords', exist_ok=True)

# Preprocessing function 
def load_and_preprocess_video(video_path, label, num_frames=32, frame_size=224):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while len(frames) < num_frames:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (frame_size, frame_size))
        frames.append(frame)
    cap.release()
    while len(frames) < num_frames:
        frames.append(frames[-1] if frames else np.zeros((frame_size, frame_size, 3), dtype=np.uint8))
    frames = np.stack(frames)
    frames = preprocess_input(frames.astype(np.float32))
    return frames, label

# Convert and store data to TFRecords 
def create_tfrecords(video_paths, labels, filename):
    def serialize_example(frames, label):
        feature = {
            'frames': tf.train.Feature(bytes_list=tf.train.BytesList(value=[
                tf.io.serialize_tensor(frames).numpy()])),
            'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))
        }
        return tf.train.Example(features=tf.train.Features(feature=feature)).SerializeToString()

    with tf.io.TFRecordWriter(f'tfrecords/{filename}.tfrecord') as writer:  # Fixed from TFRecordDataset
        for path, label in zip(video_paths, labels):
            frames, label = load_and_preprocess_video(path, label)
            example = serialize_example(frames, label)
            writer.write(example)

# Load dataset from TFRecords
def load_tfrecords(filename):
    def parse_tfrecord(example):
        feature_description = {
            'frames': tf.io.FixedLenFeature([], tf.string),
            'label': tf.io.FixedLenFeature([], tf.int64)
        }
        example = tf.io.parse_single_example(example, feature_description)
        frames = tf.io.parse_tensor(example['frames'], out_type=tf.float32)
        return frames, example['label']

    return tf.data.TFRecordDataset(f'tfrecords/{filename}.tfrecord') \
                 .map(parse_tfrecord) \
                 .batch(4) \
                 .prefetch(tf.data.AUTOTUNE)




In [ ]:
# Check if preprocessed data exists (adjusted for Colab)
if not os.path.exists('tfrecords/train.tfrecord'):
    # Upload and extract videos in Colab
    uploaded = files.upload()
    zip_filename = list(uploaded.keys())[0]
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall('videos')



Saving Sober.zip to Sober.zip
Saving Drunk.zip to Drunk.zip


In [17]:
    drunk_videos = glob('/content/drive/MyDrive/Colab Notebooks/Drunk-20250406T071554Z-001/Drunk*.mp4')
    sober_videos = glob('/content/drive/MyDrive/Colab Notebooks/Sober-20250406T071555Z-001/Sober*.mp4')
    video_paths = drunk_videos + sober_videos
    labels = [1]*len(drunk_videos) + [0]*len(sober_videos)

    # Shuffle and split
    indices = np.random.permutation(len(video_paths))
    video_paths = [video_paths[i] for i in indices]
    labels = [labels[i] for i in indices]
    train_size = int(0.8 * len(video_paths))

    # Create TFRecords
    create_tfrecords(video_paths[:train_size], labels[:train_size], 'train')
    create_tfrecords(video_paths[train_size:], labels[train_size:], 'test')

# Load datasets
train_dataset = load_tfrecords('train')
test_dataset = load_tfrecords('test')

# Model setup
cnn = ResNet50(include_top=False, pooling='avg')
cnn.trainable = False

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(32, 224, 224, 3)),
    tf.keras.layers.TimeDistributed(cnn),
    tf.keras.layers.LSTM(256),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Checkpoint setup (fixed filepath to .keras)
checkpoint_path = "checkpoints/cp-{epoch:04d}.keras"  # Changed from .ckpt
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_path,
        save_weights_only=False,  # Saves full model
        save_best_only=True,
        monitor='val_accuracy',
        verbose=1
    ),
    tf.keras.callbacks.BackupAndRestore(
        backup_dir='backup/',
        save_freq='epoch'
    ),
    tf.keras.callbacks.EarlyStopping(
        patience=5,
        restore_best_weights=True
    )
]

# Resume training if checkpoint exists
initial_epoch = 0
latest_checkpoint = tf.train.latest_checkpoint('checkpoints/')
if latest_checkpoint:
  model = tf.keras.models.load_model(latest_checkpoint)
  initial_epoch = int(latest_checkpoint.split('-')[-1].split('.')[0])



# Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

In [23]:
# 1. Import
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense

# 2. Prepare data (example)
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

# 3. Batch and prefetch
train_dataset = train_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

# 4. Build model with proper input shape
model = Sequential([
    Flatten(input_shape=(28, 28)),  # MNIST shape example
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

# 5. Compile
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 6. Train
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=10
)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.8092 - loss: 7.7707 - val_accuracy: 0.8943 - val_loss: 0.4933
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8945 - loss: 0.4571 - val_accuracy: 0.9188 - val_loss: 0.3593
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9198 - loss: 0.3353 - val_accuracy: 0.9263 - val_loss: 0.3011
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9300 - loss: 0.2802 - val_accuracy: 0.9209 - val_loss: 0.3279
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9356 - loss: 0.2531 - val_accuracy: 0.9361 - val_loss: 0.3183
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9410 - loss: 0.2309 - val_accuracy: 0.9332 - val_loss: 0.3041
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.9441 - loss: 0.2206 - val_accuracy: 0.9411 - val_loss: 0.3023
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9473 - loss: 0.2078 - val_accurac

In [33]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Reshape

# 1. CORRECT Model Architecture (for single frames)
model = Sequential([
    # Input shape for single frames (28, 28, 1)
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 2. UPDATED Prediction Function
def predict_video(video_path):
    # Load video (returns shape [32, height, width, channels])
    frames, _ = load_and_preprocess_video(video_path, 0)

    # Process each frame individually
    predictions = []
    for frame in frames:
        # Resize and convert to grayscale
        processed_frame = tf.image.resize(frame, [28, 28])
        processed_frame = tf.image.rgb_to_grayscale(processed_frame)

        # Add batch dimension and predict
        frame_pred = model.predict(processed_frame[np.newaxis, ...])[0][0]
        predictions.append(frame_pred)

    # Aggregate predictions (average across frames)
    avg_prediction = np.mean(predictions)
    return f"Drunken ({avg_prediction:.2%})" if avg_prediction > 0.5 else f"Sober ({1-avg_prediction:.2%})"

# 3. Usage
video_path = input("Enter video path: ").strip()
print(predict_video(video_path))

Enter video path: /content/drive/MyDrive/drunk.mp4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/ste